In [52]:
import datetime
import time
from esa_snappy import ProductIO, HashMap, GPF, ProductUtils,jpy
import os, gc
from shapely import wkt
import geopandas as gpd
import json
from esa_snappy import GPF

In [53]:
metadata='/data/ksa/01_Image_Acquisition/05_Json_Coverage/32_coverage_ASF.json'
with open(metadata,'r') as f:
    dt_prov=json.load(f)

In [54]:
for i in range(0,len(dt_prov)):
    print(dt_prov[i]['periode'][0])
    break

{'start_periode': '2021-01-01', 'end_periode': '2021-01-12', 'image_asf': ['S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B', 'S1A_IW_GRDH_1SDV_20210104T223357_20210104T223424_035994_043783_678D', 'S1A_IW_GRDH_1SDV_20210104T223332_20210104T223357_035994_043783_3182'], 'coverage': 99.99999999999999, 'alasan': 'Sudah OK!'}


In [63]:
def do_apply_orbit_file(source):
    print('\tApply orbit file...')
    parameters = HashMap()
    parameters.put('Apply-Orbit-File', True)
    parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
    parameters.put('continueOnFail', True)
    output = GPF.createProduct('Apply-Orbit-File', parameters, source)
    return output

def do_thermal_noise_removal(source):
    print('\tThermal noise removal...')
    parameters = HashMap()
    parameters.put('removeThermalNoise', True)
    output = GPF.createProduct('ThermalNoiseRemoval', parameters, source)
    return output

def do_remove_grd_border_noise(source):
    print('\tRemove GRD border noise...')
    parameters = HashMap()
    parameters.put('Remove-GRD-Border-Noise', True)
    parameters.put('trimThreshold', 0.5)
    output = GPF.createProduct('Remove-GRD-Border-Noise', parameters, source)
    return output

def do_calibration(source, polarization, pols):
    print('\tCalibration...')
    parameters = HashMap()
    parameters.put('outputBetaBand', False)
    parameters.put('outputGammaBand', False)
    if polarization == 'DH':
        parameters.put('sourceBands', 'Intensity_HH,Intensity_HV')
    elif polarization == 'DV':
        parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    elif polarization == 'SH' or polarization == 'HH':
        parameters.put('sourceBands', 'Intensity_HH')
    elif polarization == 'SV':
        parameters.put('sourceBands', 'Intensity_VV')
    else:
        print("different polarization!")
    parameters.put('selectedPolarisations', pols)
    parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, source)
    return output

def do_speckle_filtering(source):
    print('\tSpeckle filtering...')
    parameters = HashMap()
    parameters.put('filter', 'Refined Lee')
    output = GPF.createProduct('Speckle-Filter', parameters, source)
    return output

def do_terrain_correction(source, downsample):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'Copernicus 30m Global DEM')
    parameters.put('mapProjection', 'EPSG:3857')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('saveProjectedLocalIncidenceAngle', False)
    parameters.put('saveSelectedSourceBand', True)
    if downsample == 1:
        parameters.put('pixelSpacingInMeter', 20.0)
    output = GPF.createProduct('Terrain-Correction', parameters, source)
    return output

def lineartodb(source):
    print('\tLinear to DB Conversion...')
    parameters = HashMap()
    output = GPF.createProduct('LinearToFromdB', parameters, source)
    return output

In [64]:
image_in='/data/ksa/01_Image_Acquisition/01_Raw_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.zip'

In [65]:
def preprocessing(image_in):
    print("--------------------------------------------")
    print('Preprocessing Begin')
    gc.enable()
    gc.collect()
    sentinel_1 = ProductIO.readProduct(image_in)
    print(sentinel_1)
    loopstarttime=str(datetime.datetime.now())
    print('Start time:', loopstarttime)
    start_time = time.time()
    filename=image_in.split('/')[-1].split('_')
    image_out=image_in.replace('01_Raw_Image','02_Processed_Image_rev').replace('.zip','')
    polarization=filename[3][2:]
    if polarization == 'DV':
        pols = 'VH,VV'
    elif polarization == 'DH':
        pols = 'HH,HV'
    elif polarization == 'SH' or polarization == 'HH':
        pols = 'HH'
    elif polarization == 'SV':
        pols = 'VV'
    else:
        print("Polarization error!")
    applyorbit = do_apply_orbit_file(sentinel_1)
    thermaremoved = do_thermal_noise_removal(applyorbit)
    grdborder = do_remove_grd_border_noise(thermaremoved)
    calibrated = do_calibration(grdborder, polarization, pols)
    down_speckled=do_speckle_filtering(calibrated)
    down_corrected=do_terrain_correction(down_speckled,1)
    convert=lineartodb(down_corrected)
    print("Writing...")
    ProductIO.writeProduct(convert, image_out,'BEAM-DIMAP')
    del applyorbit
    del thermaremoved
    del grdborder
    del calibrated
    del down_speckled
    del down_corrected
    sentinel_1.dispose()
    sentinel_1.closeIO()
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Finshed)
    print("--------------------------------------------")

In [58]:
def create_folder_if_not_exist(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

In [81]:
def mosaicing(list_sources,bound,wkt_bounds,outflnm):
    print('==========================================================')
    print('Mosaicking Process Begin')
    loopstarttime=str(datetime.datetime.now())
    print('Start time:', loopstarttime)
    start_time = time.time()
    products = jpy.array('org.esa.snap.core.datamodel.Product',len(list_sources))
    Variable = jpy.get_type('org.esa.snap.core.gpf.common.MosaicOp$Variable')
    variables = jpy.array('org.esa.snap.core.gpf.common.MosaicOp$Variable', 2)
    
    parameters = HashMap()
    parameters.put('combine', 'OR')
    parameters.put('crs', 'EPSG:3857')
    parameters.put('resampling', 'Nearest')
    parameters.put('pixelSizeX', 20.0)
    parameters.put('pixelSizeY', 20.0)
    parameters.put('orthorectify', False)
    parameters.put('westBound',bound[0])
    parameters.put('southBound',bound[1])
    parameters.put('northBound',bound[3])
    parameters.put('eastBound',bound[2])
    
    for i in range(len(list_sources)):
        p=ProductIO.readProduct('/data/ksa/01_Image_Acquisition/02_Processed_Image_rev/'+list_sources[i]+'.dim')
        products[i]=p
    if len(list_sources)>0:
        band_names = products[0].getBandNames()
        i=0
        for band in band_names:
            print(band)
            variables[i]=Variable(band,band)
            i=i+1 
    parameters.put('variables', variables) 
    output = GPF.createProduct('Mosaic', parameters, products)
    
    parameters = HashMap()
    parameters.put('geoRegion', wkt_bounds)
    output = GPF.createProduct('Subset', parameters, output)

    ProductIO.writeProduct(output, outflnm, 'BEAM-DIMAP')
    for i in range(len(list_sources)):
        products[i].dispose()
        products[i].closeIO()
    del output
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Finished")
    print('==========================================================')
    

In [82]:
def run_mosaic(dict_mosaic):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('PREPROCESSING FOLLOWED BY MOSAICKING FOR ID:',dict_mosaic['id'])
    create_folder_if_not_exist('/data/ksa/01_Image_Acquisition/02_Processed_mosaic/'+dict_mosaic['id'])
    #create_folder_if_not_exist('/data/ksa/01_Image_Acquisition/02_Processed_mosaic/'+dict_mosaic['id']+'/temp')
    bounds=gpd.GeoDataFrame(geometry=[wkt.loads(dict_mosaic['geometry'])],crs=4326)#.to_crs(3857)
    wkt_bounds=bounds.geometry.to_wkt()[0]
    total_bounds=bounds.total_bounds
    period_dict=dict_mosaic['periode']
    for i in range(len(period_dict)):
        start_periode=period_dict[i]['start_periode']
        end_periode=period_dict[i]['end_periode']
        outflnm='/data/ksa/01_Image_Acquisition/02_Processed_mosaic/'+dict_mosaic['id']+'/'+start_periode.replace('-','')+'_'+end_periode.replace('-','')
        list_sources= period_dict[i]['image_asf']
        for j in list_sources:
            if not os.path.exists('/data/ksa/01_Image_Acquisition/02_Processed_Image_rev/'+j+'.dim'):
                preprocessing('/data/ksa/01_Image_Acquisition/01_Raw_Image/'+j+'.zip')
        mosaicing(list_sources,total_bounds,wkt_bounds,outflnm)
        break
    print('FINISHED')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')    

In [83]:
run_mosaic(dt_prov[0])

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PREPROCESSING FOLLOWED BY MOSAICKING FOR ID: 48MXU
Folder '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU' already exists.
Mosaicking Process Begin
Start time: 2024-08-05 20:14:01.808021
Sigma0_VH_db
Sigma0_VV_db

100% done.

100% done.

100% done.

100% done.

100% done.
--- 5.759768962860107 seconds ---
FINISHED
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [85]:
import zipfile

zf = zipfile.ZipFile("myzipfile.zip", "w")
for dirname, subdirs, files in os.walk("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()